In [ ]:
# =============================================================================
# CELL 1: IMPORTS
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score, mean_squared_error

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Imports complete")

In [ ]:
# =============================================================================
# CELL 2: CONFIGURATION - MODIFY THESE
# =============================================================================

FILE_PATH = "your_data.xlsx"           # <-- Your Excel file
RESPONSE_COLUMN = "yield"              # <-- Your response variable name  
EXCLUDE_COLUMNS = ["experiment_id"]    # <-- Columns to exclude from features
TEST_SIZE = 0.2
TOP_K = 5

print(f"Response: {RESPONSE_COLUMN}")
print(f"Exclude: {EXCLUDE_COLUMNS}")
print(f"Top K: {TOP_K}")

In [ ]:
# =============================================================================
# CELL 3: LOAD DATA
# =============================================================================

# Try loading your file, or create sample data for demo
try:
    df = pd.read_excel(FILE_PATH)
    print(f"✓ Loaded: {df.shape}")
except:
    print("Creating sample chemical data...")
    np.random.seed(42)
    n = 200
    df = pd.DataFrame({
        'experiment_id': range(1, n+1),
        'temperature_C': np.random.uniform(20, 100, n),
        'pressure_bar': np.random.uniform(1, 10, n),
        'pH': np.random.uniform(2, 12, n),
        'concentration_mol_L': np.random.exponential(0.5, n),
        'reaction_time_min': np.random.uniform(5, 120, n),
        'catalyst_amount_g': np.random.uniform(0.1, 5, n),
        'stirring_speed_rpm': np.random.uniform(100, 1000, n),
        'solvent_ratio': np.random.uniform(0.1, 0.9, n),
        'humidity_percent': np.random.uniform(30, 80, n),
        'particle_size_um': np.random.uniform(1, 100, n),
    })
    # Response with known relationships
    df['yield'] = (
        0.5 * df['temperature_C'] +
        2.0 * df['catalyst_amount_g'] +
        -0.3 * df['pH'] +
        0.1 * df['reaction_time_min'] +
        np.random.normal(0, 5, n)
    ).clip(0, 100)
    
    # Add some missing values
    df.loc[np.random.choice(n, 5), 'humidity_percent'] = np.nan
    print(f"✓ Sample data created: {df.shape}")

print(f"\nColumns: {list(df.columns)}")
print(f"\nMissing values:\n{df.isnull().sum()[df.isnull().sum() > 0]}")
df.head()

In [ ]:
# =============================================================================
# CELL 4: PREPARE FEATURES AND TARGET
# =============================================================================

# Get numeric columns only, exclude specified columns and response
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [c for c in numeric_cols if c not in EXCLUDE_COLUMNS + [RESPONSE_COLUMN]]

X = df[feature_cols].copy()
y = df[RESPONSE_COLUMN].copy()

print(f"Features ({len(feature_cols)}): {feature_cols}")
print(f"Target: {RESPONSE_COLUMN}")
print(f"Shape: X={X.shape}, y={y.shape}")

In [ ]:
# =============================================================================
# CELL 5: CHEMICAL DATA CHECKS
# =============================================================================

# Check for outliers (Z-score > 3)
print("=" * 50)
print("OUTLIER DETECTION (Z-score > 3)")
print("=" * 50)
z_scores = np.abs((X - X.mean()) / X.std())
outliers = (z_scores > 3).sum()
for col in outliers[outliers > 0].index:
    print(f"  {col}: {outliers[col]} outliers")
if outliers.sum() == 0:
    print("  No outliers detected")

# Check multicollinearity
print("\n" + "=" * 50)
print("MULTICOLLINEARITY CHECK (|r| > 0.9)")
print("=" * 50)
corr = X.corr().abs()
high_corr = []
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        if corr.iloc[i, j] > 0.9:
            high_corr.append((corr.columns[i], corr.columns[j], corr.iloc[i, j]))
            print(f"  {corr.columns[i]} ↔ {corr.columns[j]}: {corr.iloc[i,j]:.3f}")
if not high_corr:
    print("  No highly correlated features")

# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(X.corr(), annot=True, cmap='RdBu_r', center=0, fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# CELL 6: TRAIN/TEST SPLIT (BEFORE STANDARDIZATION - NO LEAKAGE)
# =============================================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

In [ ]:
# =============================================================================
# CELL 7: IMPUTE MISSING VALUES (FIT ON TRAIN ONLY)
# =============================================================================

imputer = SimpleImputer(strategy='median')

# Fit on train, transform both
X_train = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_test = pd.DataFrame(
    imputer.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# Handle missing in y
train_mask = ~y_train.isnull()
test_mask = ~y_test.isnull()
X_train, y_train = X_train[train_mask], y_train[train_mask]
X_test, y_test = X_test[test_mask], y_test[test_mask]

print(f"✓ Missing values imputed (fitted on training only)")
print(f"  Final training: {len(X_train)}, test: {len(X_test)}")

In [ ]:
# =============================================================================
# CELL 8: STANDARDIZATION (FIT ON TRAIN ONLY - NO LEAKAGE)
# =============================================================================

scaler = StandardScaler()

# Fit on training data ONLY
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

# Apply training statistics to test data
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

print("✓ Standardization complete (NO DATA LEAKAGE)")
print("  - Scaler fitted on training data only")
print("  - Training mean applied to test data")
print(f"\nTraining data stats (should be ~0 mean, ~1 std):")
print(X_train_scaled.describe().loc[['mean', 'std']].round(4))

In [ ]:
# =============================================================================
# CELL 9: METHOD 1 - LINEAR REGRESSION
# =============================================================================

print("=" * 50)
print("METHOD 1: LINEAR REGRESSION")
print("=" * 50)

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Coefficients as importance (absolute value for standardized data)
lr_importance = pd.DataFrame({
    'feature': X_train_scaled.columns,
    'coefficient': lr_model.coef_,
    'importance': np.abs(lr_model.coef_)
}).sort_values('importance', ascending=False).reset_index(drop=True)
lr_importance['rank'] = range(1, len(lr_importance) + 1)

# Performance
lr_train_r2 = r2_score(y_train, lr_model.predict(X_train_scaled))
lr_test_r2 = r2_score(y_test, lr_model.predict(X_test_scaled))

print(f"\nTrain R²: {lr_train_r2:.4f}")
print(f"Test R²:  {lr_test_r2:.4f}")
print(f"\nFeature Importance (|coefficient|):")
print(lr_importance[['rank', 'feature', 'coefficient', 'importance']].to_string(index=False))

# Plot
plt.figure(figsize=(10, 6))
colors = ['green' if c > 0 else 'red' for c in lr_importance['coefficient']]
plt.barh(lr_importance['feature'][::-1], lr_importance['importance'][::-1], color=colors[::-1])
plt.xlabel('|Coefficient|')
plt.title('Linear Regression Feature Importance\n(Green=Positive, Red=Negative)')
plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# CELL 10: METHOD 2 - RANDOM FOREST
# =============================================================================

print("=" * 50)
print("METHOD 2: RANDOM FOREST")
print("=" * 50)

rf_model = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
rf_model.fit(X_train_scaled, y_train)

# Built-in feature importance (MDI)
rf_importance = pd.DataFrame({
    'feature': X_train_scaled.columns,
    'importance_mdi': rf_model.feature_importances_
}).sort_values('importance_mdi', ascending=False).reset_index(drop=True)

# Permutation importance (more reliable)
print("\nCalculating permutation importance...")
perm_result = permutation_importance(
    rf_model, X_test_scaled, y_test,
    n_repeats=30, random_state=RANDOM_STATE, n_jobs=-1
)
rf_importance['importance_perm'] = [
    perm_result.importances_mean[list(X_train_scaled.columns).index(f)] 
    for f in rf_importance['feature']
]
rf_importance['rank'] = range(1, len(rf_importance) + 1)

# Performance
rf_train_r2 = r2_score(y_train, rf_model.predict(X_train_scaled))
rf_test_r2 = r2_score(y_test, rf_model.predict(X_test_scaled))

print(f"\nTrain R²: {rf_train_r2:.4f}")
print(f"Test R²:  {rf_test_r2:.4f}")
print(f"\nFeature Importance:")
print(rf_importance[['rank', 'feature', 'importance_mdi', 'importance_perm']].to_string(index=False))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].barh(rf_importance['feature'][::-1], rf_importance['importance_mdi'][::-1], color='forestgreen')
axes[0].set_xlabel('Importance (MDI)')
axes[0].set_title('Random Forest - Mean Decrease Impurity')

rf_perm_sorted = rf_importance.sort_values('importance_perm', ascending=False)
axes[1].barh(rf_perm_sorted['feature'][::-1], rf_perm_sorted['importance_perm'][::-1], color='steelblue')
axes[1].set_xlabel('Importance (Permutation)')
axes[1].set_title('Random Forest - Permutation Importance')
plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# CELL 11: METHOD 3 - GAUSSIAN PROCESS (BAYESIAN)
# =============================================================================

print("=" * 50)
print("METHOD 3: GAUSSIAN PROCESS (BAYESIAN)")
print("=" * 50)

# Subsample if needed (GP is O(n³))
max_samples = 500
if len(X_train_scaled) > max_samples:
    print(f"Subsampling to {max_samples} for GP (computational efficiency)")
    idx = np.random.choice(len(X_train_scaled), max_samples, replace=False)
    X_train_gp = X_train_scaled.iloc[idx]
    y_train_gp = y_train.iloc[idx]
else:
    X_train_gp = X_train_scaled
    y_train_gp = y_train

# Kernel with ARD
n_features = X_train_scaled.shape[1]
kernel = ConstantKernel(1.0) * Matern(length_scale=np.ones(n_features), nu=2.5) + WhiteKernel(1.0)

print("Fitting Gaussian Process (may take a moment)...")
gp_model = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=5,
    random_state=RANDOM_STATE,
    normalize_y=True
)
gp_model.fit(X_train_gp, y_train_gp)

# Permutation importance
print("Calculating permutation importance...")
gp_perm = permutation_importance(
    gp_model, X_test_scaled, y_test,
    n_repeats=10, random_state=RANDOM_STATE, n_jobs=-1
)

gp_importance = pd.DataFrame({
    'feature': X_train_scaled.columns,
    'importance': gp_perm.importances_mean,
    'importance_std': gp_perm.importances_std
}).sort_values('importance', ascending=False).reset_index(drop=True)
gp_importance['rank'] = range(1, len(gp_importance) + 1)

# Performance
gp_train_r2 = r2_score(y_train_gp, gp_model.predict(X_train_gp))
gp_test_r2 = r2_score(y_test, gp_model.predict(X_test_scaled))

print(f"\nTrain R²: {gp_train_r2:.4f}")
print(f"Test R²:  {gp_test_r2:.4f}")
print(f"\nFeature Importance (Permutation):")
print(gp_importance[['rank', 'feature', 'importance', 'importance_std']].to_string(index=False))

# Plot
plt.figure(figsize=(10, 6))
plt.barh(gp_importance['feature'][::-1], gp_importance['importance'][::-1], 
         xerr=gp_importance['importance_std'][::-1], color='darkorange', capsize=3)
plt.xlabel('Permutation Importance')
plt.title('Gaussian Process - Feature Importance')
plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# CELL 12: AGGREGATE RANKINGS
# =============================================================================

print("=" * 50)
print("AGGREGATED RANKINGS (ALL 3 METHODS)")
print("=" * 50)

# Create ranking from each method
lr_ranking = lr_importance[['feature']].copy()
lr_ranking['rank_lr'] = range(1, len(lr_ranking) + 1)

rf_ranking = rf_importance[['feature']].copy()
rf_ranking['rank_rf'] = range(1, len(rf_ranking) + 1)

gp_ranking = gp_importance[['feature']].copy()
gp_ranking['rank_gp'] = range(1, len(gp_ranking) + 1)

# Merge
combined = lr_ranking.merge(rf_ranking, on='feature').merge(gp_ranking, on='feature')

# Average rank (lower = more important)
combined['avg_rank'] = combined[['rank_lr', 'rank_rf', 'rank_gp']].mean(axis=1)
combined = combined.sort_values('avg_rank').reset_index(drop=True)
combined['final_rank'] = range(1, len(combined) + 1)

print("\nConsensus Ranking:")
print(combined[['final_rank', 'feature', 'rank_lr', 'rank_rf', 'rank_gp', 'avg_rank']].to_string(index=False))

# Get top K
top_k_features = combined.head(TOP_K)['feature'].tolist()
print(f"\n✓ TOP {TOP_K} FEATURES: {top_k_features}")

# Store for later use
consensus_ranking = combined.copy()

In [ ]:
# =============================================================================
# CELL 13: RANKING VISUALIZATION
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Bar comparison
ax1 = axes[0]
x = np.arange(len(combined.head(TOP_K)))
width = 0.25

ax1.bar(x - width, combined.head(TOP_K)['rank_lr'], width, label='Linear Reg', color='steelblue')
ax1.bar(x, combined.head(TOP_K)['rank_rf'], width, label='Random Forest', color='forestgreen')
ax1.bar(x + width, combined.head(TOP_K)['rank_gp'], width, label='Gaussian Process', color='darkorange')

ax1.set_xlabel('Feature')
ax1.set_ylabel('Rank (lower = better)')
ax1.set_title('Ranking Comparison - Top Features')
ax1.set_xticks(x)
ax1.set_xticklabels(combined.head(TOP_K)['feature'], rotation=45, ha='right')
ax1.legend()
ax1.invert_yaxis()

# Heatmap
ax2 = axes[1]
heatmap_data = combined.set_index('feature')[['rank_lr', 'rank_rf', 'rank_gp']]
heatmap_data.columns = ['Linear Reg', 'Random Forest', 'Gaussian Process']
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='RdYlGn_r', ax=ax2)
ax2.set_title('Feature Rankings Heatmap\n(Lower = More Important)')

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# CELL 14: MANUAL FEATURE SELECTION (FOR RESEARCHERS TO OVERRIDE)
# =============================================================================

def select_features(features_to_use):
    """
    Manually select which features to use.
    Call this function with your desired feature list.
    
    Parameters:
    -----------
    features_to_use : list
        List of feature names to use
        
    Returns:
    --------
    selected_features : list
        Validated list of selected features
        
    Example:
    --------
    selected = select_features(['temperature_C', 'catalyst_amount_g', 'pH'])
    """
    all_features = list(X_train_scaled.columns)
    
    # Validate
    invalid = [f for f in features_to_use if f not in all_features]
    valid = [f for f in features_to_use if f in all_features]
    
    if invalid:
        print(f"⚠ Invalid features (ignored): {invalid}")
    
    print(f"\n✓ Selected {len(valid)} features:")
    for i, f in enumerate(valid, 1):
        # Find ranks from each method
        lr_rank = lr_importance[lr_importance['feature'] == f]['rank'].values[0]
        rf_rank = rf_importance[rf_importance['feature'] == f]['rank'].values[0]
        gp_rank = gp_importance[gp_importance['feature'] == f]['rank'].values[0]
        print(f"  {i}. {f} (LR:{lr_rank}, RF:{rf_rank}, GP:{gp_rank})")
    
    return valid


def show_all_features():
    """Display all available features with their rankings."""
    print("\nALL AVAILABLE FEATURES:")
    print("-" * 70)
    print(f"{'#':<4} {'Feature':<25} {'LR':<6} {'RF':<6} {'GP':<6} {'Avg':<6}")
    print("-" * 70)
    
    for _, row in consensus_ranking.iterrows():
        print(f"{int(row['final_rank']):<4} {row['feature']:<25} "
              f"{int(row['rank_lr']):<6} {int(row['rank_rf']):<6} "
              f"{int(row['rank_gp']):<6} {row['avg_rank']:<6.2f}")


def use_top_k(k):
    """Select top K features from consensus ranking."""
    features = consensus_ranking.head(k)['feature'].tolist()
    print(f"✓ Selected top {k} features: {features}")
    return features


# Show options
show_all_features()
print(f"\nCurrent selection (top {TOP_K}): {top_k_features}")

In [ ]:
# =============================================================================
# CELL 15: EXAMPLE - OVERRIDE FEATURE SELECTION
# =============================================================================

# OPTION 1: Use top K from consensus
selected_features = use_top_k(TOP_K)

# OPTION 2: Manually select specific features (uncomment to use)
# selected_features = select_features(['temperature_C', 'catalyst_amount_g', 'pH', 'reaction_time_min'])

# OPTION 3: Select by examining rankings and choosing manually
# show_all_features()  # Look at rankings
# selected_features = select_features(['your', 'chosen', 'features'])

print(f"\n{'='*50}")
print(f"FINAL SELECTED FEATURES: {selected_features}")
print(f"{'='*50}")

In [ ]:
# =============================================================================
# CELL 16: FINAL MODEL WITH SELECTED FEATURES
# =============================================================================

# Subset data to selected features
X_train_final = X_train_scaled[selected_features]
X_test_final = X_test_scaled[selected_features]

print("=" * 50)
print("FINAL MODEL COMPARISON (Selected Features Only)")
print("=" * 50)
print(f"Features used: {selected_features}\n")

# Linear Regression
lr_final = LinearRegression()
lr_final.fit(X_train_final, y_train)
lr_r2 = r2_score(y_test, lr_final.predict(X_test_final))
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_final.predict(X_test_final)))

# Random Forest
rf_final = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
rf_final.fit(X_train_final, y_train)
rf_r2 = r2_score(y_test, rf_final.predict(X_test_final))
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_final.predict(X_test_final)))

# Gaussian Process
gp_kernel = ConstantKernel(1.0) * Matern(length_scale=np.ones(len(selected_features)), nu=2.5) + WhiteKernel(1.0)
gp_final = GaussianProcessRegressor(kernel=gp_kernel, n_restarts_optimizer=5, random_state=RANDOM_STATE, normalize_y=True)
gp_final.fit(X_train_final, y_train)
gp_r2 = r2_score(y_test, gp_final.predict(X_test_final))
gp_rmse = np.sqrt(mean_squared_error(y_test, gp_final.predict(X_test_final)))

# Results
results = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'Gaussian Process'],
    'Test R²': [lr_r2, rf_r2, gp_r2],
    'Test RMSE': [lr_rmse, rf_rmse, gp_rmse]
})
print(results.to_string(index=False))

# Plot predictions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
models = [('Linear Regression', lr_final), ('Random Forest', rf_final), ('Gaussian Process', gp_final)]

for ax, (name, model) in zip(axes, models):
    y_pred = model.predict(X_test_final)
    ax.scatter(y_test, y_pred, alpha=0.6)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    ax.set_xlabel('Actual')
    ax.set_ylabel('Predicted')
    ax.set_title(f'{name}\nR² = {r2_score(y_test, y_pred):.4f}')

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# CELL 17: EXPORT RESULTS
# =============================================================================

# Save rankings to CSV
consensus_ranking.to_csv('feature_rankings.csv', index=False)
print("✓ Saved: feature_rankings.csv")

# Save selected features
with open('selected_features.txt', 'w') as f:
    f.write('\n'.join(selected_features))
print("✓ Saved: selected_features.txt")

# Summary
print("\n" + "=" * 50)
print("SUMMARY")
print("=" * 50)
print(f"Total features analyzed: {len(feature_cols)}")
print(f"Selected features: {len(selected_features)}")
print(f"Features: {selected_features}")
print(f"Best model: {results.loc[results['Test R²'].idxmax(), 'Model']}")
print(f"Best R²: {results['Test R²'].max():.4f}")